In [10]:
import numpy as np
import pandas as pd
import spacy

In [4]:
import spacy

In [5]:
from spacy.tokenizer import Tokenizer
import nltk

In [6]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

In [7]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [37]:
from spacy.vocab import Vocab
from sklearn.model_selection import train_test_split


In [11]:
df_train = pd.read_csv("c:/data/kaggle/Disaster tweets/train.csv")

In [12]:
df_train.isna().sum(axis=0)

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [13]:
df_train[df_train["keyword"].isnull()==False]

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0


We have three columns to use to predict on, keyword and text are nlp, location could help but leave for now

In [14]:
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

In [15]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [16]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [17]:
example = nlp(df_train["text"][1])

In [18]:
example.vector

array([ 1.0309792 , -1.2418449 , -0.70834446,  2.372108  ,  2.240989  ,
        2.5972116 , -0.01759425,  0.5596446 ,  1.4090172 ,  1.6113966 ,
        1.5328721 , -0.9112137 ,  0.47672936, -1.4448929 , -0.9597081 ,
       -1.3382363 ,  1.2704474 ,  1.4045588 ,  1.0939984 , -0.39732674,
       -0.25665942, -1.0874226 ,  1.1145511 , -1.4253209 , -0.56490976,
        0.07184009,  0.27531934, -0.9448942 ,  0.34717762, -1.9551909 ,
       -0.1868523 ,  0.5933187 ,  1.204904  ,  0.37316045,  0.93774116,
       -2.2168355 ,  3.102009  , -1.7005    , -0.8508695 , -0.5461243 ,
        1.9598846 , -0.17920429,  0.44159058, -2.3268094 , -0.7807982 ,
       -1.1765788 ,  0.90138495, -0.9002015 , -0.4945909 , -0.31053415,
       -0.46314704, -2.1820452 , -0.547442  , -0.58392787, -3.1788797 ,
        0.4143385 ,  0.5025425 ,  2.1572542 , -0.569477  ,  0.11471212,
       -0.52791154, -0.9969019 ,  0.43371034,  0.24727356, -0.5470768 ,
        0.6669565 ,  0.4905461 , -1.7539945 , -0.9240975 ,  1.54

In [32]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations ]
    
    # Removing @person from tweets
    mytokens = [word for word in mytokens if word[0:1] != "@"]
    
    # Removing links
    mytokens = [word for word in mytokens if word[0:4] != "http"]
    
    mytokens = [word for word in mytokens if word[0:2] != ".."]

    # return preprocessed list of tokens
    return mytokens

In [33]:
for i in range(0, 100,5):
    print(i)
    print(spacy_tokenizer(df_train["text"][i]))
    print(df_train["target"][i])
    print("------------")

0
['deeds', 'reason', 'earthquake', 'allah', 'forgive']
1
------------
5
['rockyfire', 'update', 'california', 'hwy', '20', 'closed', 'directions', 'lake', 'county', 'fire', 'cafire', 'wildfires']
1
------------
10
['people', 'died', 'heat', 'wave', 'far']
1
------------
15
['man']
0
------------
20
['ridiculous']
0
------------
25
['way', 'eat', 'shit']
0
------------
30
['end']
0
------------
35
['plus', 'look', 'sky', 'night', 'ablaze']
0
------------
40
['check', 'nsfw']
0
------------
45
['gained', '3', 'followers', 'week', 'know', 'stats', 'grow']
0
------------
50
['deputies', 'man', 'shot', 'brighton', 'home', 'set', 'ablaze']
1
------------
55
['kurds', 'trampling', 'turkmen', 'flag', 'later', 'set', 'ablaze', 'vandalized', 'offices', 'turkmen', 'diyala']
1
------------
60
['revel', 'wmv', 'videos', 'means', 'mac', 'farewell', 'ablaze', 'wmv', 'en', 'route', 'dvd', 'gtxrwm']
0
------------
65
['huge', 'fire', 'wholesale', 'markets', 'ablaze']
1
------------
70
['personalinjury

In [44]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    return text.strip().lower()

In [34]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [36]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [39]:
X = df_train["text"]
y= df_train["target"]

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [41]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [45]:
# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x00000228E023C7F0>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x00000228DF49DB70>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_

In [46]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.8077933450087565
Logistic Regression Precision: 0.8197747183979975
Logistic Regression Recall: 0.6894736842105263


In [48]:
from sklearn.metrics import confusion_matrix

In [49]:
confusion_matrix(y_test, predicted)

array([[1190,  144],
       [ 295,  655]], dtype=int64)

In [34]:
tokens = Tokenizer(Vocab(df_train["text"]))

In [37]:
tfidf_matrix = TfidfVectorizer(tokens)

In [41]:
tfidf_matrix

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input=<spacy.tokenizer.Tokenizer object at 0x000001A7FD371A98>,
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
